In [ ]:
###  此脚本适用于Autodl云平台部署WavLip-GFPGAN算法
####  本脚本置于根目录下即可
####  运行本脚本前，需要在终端下载ffmpeg，命令如下：
####  `sudo apt update`
####  `sudo apt install -y ffmpeg`
####  git拉取,pip下载依赖,wget下载模型等代码只需运行第一次后就可以完全注释
####  如果模型下载有问题，请直接进入 https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/tag/AI 手动下载

In [ ]:
# 资源加速
!source /etc/network_turbo
storePath = "/root/autodl-tmp/"  # 将仓库拉取拉取到autodl的数据盘
# 进入存放目录的文件夹
%cd {storePath}

下面第一个单元格执行完一次就注释

In [ ]:
# 前缀加速
!git clone https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Wav2Lip-GFPGAN.git

In [ ]:
basePath = "/root/autodl-tmp/Wav2Lip-GFPGAN" 
# 进入项目文件夹 
%cd {basePath}

In [ ]:
# 下载依赖，如果报红就改requests，警告无所谓
!pip install -r requirements.txt  

In [ ]:
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

In [ ]:
# 下载s3fd.pth模型到指定目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/s3fd.pth -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
# 下载wav2lip.pth模型到指定目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/wav2lip.pth -O {wav2lipPath}'/checkpoints/wav2lip.pth' 

In [ ]:
import os
import cv2

outputPath = basePath+'/outputs'
inputAudioPath = basePath + '/inputs/wenan.wav'  # 改成自己的音频文件
inputVideoPath = basePath + '/inputs/tecent_female.mp4'  # 改成自己的视频文件
lipSyncedOutputPath = basePath + '/outputs/result.mp4'  # 改成输出目录，如果不想覆盖之前的视频就命个别的名

def get_video_fps(video_path):
    # 打开视频文件
    cap = cv2.VideoCapture(video_path)

    # 获取帧率
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 释放视频文件
    cap.release()

    return fps

# 获取视频帧率
fps = get_video_fps(inputVideoPath)

FPS = fps  # 设置帧率，很重要，如果不匹配会导致嘴型不匹配

if not os.path.exists(outputPath):
  os.makedirs(outputPath)
else:
    # 把存了旧temp照片的输出目录删了再建一个新的空目录
    !rm -r {outputPath}
    os.makedirs(outputPath)


In [ ]:
!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/wav2lip.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}

In [ ]:
# setup.py定义项目的元数据,包括版本信息、作者、依赖项等,并通过调用 setup() 函数将项目打包成一个可分发的 Python 包。在命令行中运行 python setup.py install 或 pip install . 将会安装该包及其相关依赖
!cd $gfpganFolderName && python setup.py develop  

In [ ]:

### 这里注意自己的python版本号
# 下载GFPGANv1.3.pth模型到预训练模型文件夹目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'
# 下载GFPGANv1.4.pth模型到预训练模型文件夹目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/GFPGANv1.4.pth -P {gfpganFolderName}'/experiments/pretrained_models'
# 下载detection_Resnet50_Final.pth模型到权重模型文件夹目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/detection_Resnet50_Final.pth -P '/root/miniconda3/lib/python3.10/site-packages/facexlib/weights/'
# 下载parsing_parsenet.pth模型到权重模型文件夹目录
!wget https://github.moeyy.xyz/https://github.com/Wild-Piggggggg/Dreamy-umbrella/releases/download/AI/parsing_parsenet.pth -P '/root/miniconda3/lib/python3.10/site-packages/facexlib/weights/'

In [ ]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'  # 这里也要改
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


In [ ]:
!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.4 -s 2 --only_center_face --bg_upsampler None

In [ ]:
import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2

# FPS = 25  # 改成自己的帧率!!

batch = 0
batchSize = int(FPS*10)
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), FPS, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()


In [ ]:
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile=open(concatTextFilePath,"w")
for ips in range(batch):
  concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.avi"
!ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath} 

finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.avi'
!ffmpeg -y -i {concatedVideoOutputPath} -i {inputAudioPath} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}